In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from PIL import Image
import numpy as np
import pandas as pd
import timm
from sklearn.metrics import f1_score
import torch.nn.functional as F
from torch.optim.lr_scheduler import ReduceLROnPlateau
import cv2

In [ ]:
# Assuming 'tf_efficientnet_b0' is used
model = timm.create_model('tf_efficientnet_b0', pretrained=False)
NUM_FINETUNE_CLASSES = 100  # Adjust based on your specific number of classes

# Modify the classifier for fine-tuning with your specific number of classes
model.classifier = nn.Linear(model.num_features, NUM_FINETUNE_CLASSES)

In [ ]:
# https://www.kaggle.com/datasets/ericrohmer/compet/

train_features = pd.read_csv('/kaggle/input/compet/train_features.csv', header=None)
train_labels = pd.read_csv('/kaggle/input/compet/train_labels.csv', header=None)
encoder = OneHotEncoder()
train_labels_encoded = encoder.fit_transform(train_labels.values.reshape(-1, 1)).toarray()

# Standardize the features
column_names = train_features.columns.tolist()
scaler = StandardScaler()
train_features_scaled = scaler.fit_transform(train_features)
train_features = pd.DataFrame(data=train_features_scaled, columns=column_names)

In [ ]:
# Assuming train_features and train_labels_encoded are your features and labels
train_data, val_data, train_labels, val_labels = train_test_split(
    train_features, 
    train_labels_encoded, 
    test_size=0.2, 
    random_state=42,
    stratify=train_labels_encoded  # Ensure stratified splitting
)

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, features, labels, transform):
        self.features = features
        self.labels = labels
        self.transform = transform
        self.height = 224
        self.width = 224
        self.channels = 3

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        img_row = self.features.iloc[idx]
        image_array = np.array(img_row).reshape((32, 32))
        image = Image.fromarray(image_array.astype(np.uint8))
        img = image.convert('RGB')
        
        img_np = np.array(img)
        resized_img = cv2.resize(img_np, (self.width, self.height), interpolation=cv2.INTER_CUBIC)

        # Convert NumPy array back to PIL image
        resized_img_pil = Image.fromarray(resized_img)
        
        tensor = self.transform(resized_img_pil)
        label = self.labels[idx]
        return tensor, label

# Add normalization to your transformation
transform = transforms.Compose([
    transforms.ToTensor()
])

In [ ]:
# Create CustomDataset instances
train_dataset = CustomDataset(features=train_data, labels=train_labels, transform=transform)
val_dataset = CustomDataset(features=val_data, labels=val_labels, transform=transform)

# Create DataLoader for training and validation sets
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
train_labels.shape

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
y = np.argmax(train_labels, axis=1)

class_counts = np.bincount(y)
n_samples = len(y)
n_classes = len(class_counts)
class_weights = n_samples / (n_classes * class_counts)

# Convert class weights to a tensor
weights_tensor = torch.tensor(class_weights, dtype=torch.float)
weights_tensor = weights_tensor.to(device)

In [ ]:
learning_rate = 0.001
criterion = nn.CrossEntropyLoss(weight=weights_tensor)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=1e-5)

In [ ]:
# Training loop
num_epochs = 100
patience = 10  # Set the patience for early stopping

best_val_loss = float('inf')
current_patience = 0

# Move the model to GPU if available
model.to(device)

total_train = 0
correct_train = 0
total_val = 0
correct_val = 0

train_losses = []
train_accuracies = []
train_f1_scores = []
val_losses = []
val_accuracies = []
val_f1_scores = []

for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    all_labels_train = []
    all_predictions_train = []
    

    for inputs, one_hot_labels in train_loader:
        inputs, one_hot_labels = inputs.to(device), one_hot_labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        
        # Convert one-hot encoded labels to class indices
        labels = torch.argmax(one_hot_labels, dim=1).to(device)
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

        # Calculate training accuracy
        _, predicted_train = torch.max(outputs, 1)
        total_train += labels.size(0)
        correct_train += (predicted_train == labels).sum().item()

        # Collect true labels and predicted labels for F1 score
        all_labels_train.extend(labels.cpu().numpy())
        all_predictions_train.extend(predicted_train.cpu().numpy())
        
        _, predicted_train = torch.max(outputs, 1)
        total_train += labels.size(0)
        correct_train += (predicted_train == labels).sum().item()

    f1_train = f1_score(all_labels_train, all_predictions_train, average='weighted')
    train_accuracy = correct_train / total_train
        
    # Validation loop
    model.eval()
    val_loss = 0.0
    all_labels_val = []
    all_predictions_val = []

    with torch.no_grad():
        for inputs, one_hot_labels in val_loader:
            inputs, one_hot_labels = inputs.to(device), one_hot_labels.to(device)
            
            outputs = model(inputs)
            
            # Convert one-hot encoded labels to class indices
            labels = torch.argmax(one_hot_labels, dim=1).to(device)
            
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            # Calculate validation accuracy
            _, predicted_val = torch.max(outputs, 1)
            total_val += labels.size(0)
            correct_val += (predicted_val == labels).sum().item()

            # Collect true labels and predicted labels for F1 score
            all_labels_val.extend(labels.cpu().numpy())
            all_predictions_val.extend(predicted_val.cpu().numpy())

            _, predicted_val = torch.max(outputs, 1)
            total_val += labels.size(0)
            correct_val += (predicted_val == labels).sum().item()


    # Calculate average losses
    avg_train_loss = train_loss / len(train_loader.dataset)
    avg_val_loss = val_loss / len(val_loader.dataset)
    
    val_accuracy = correct_val / total_val
    f1_val = f1_score(all_labels_val, all_predictions_val, average='weighted')

    # Access the learning rate from the optimizer
    current_lr = optimizer.param_groups[0]['lr']

    # Print epoch statistics
    print(f'Epoch {epoch + 1}/{num_epochs}, '
          f'Train Loss: {avg_train_loss:.4f}, '
          f'Train Accuracy: {train_accuracy:.4f}, '
          f'Train F1 Score: {f1_train:.4f}, '
          f'Validation Loss: {avg_val_loss:.4f}, '
          f'Validation Accuracy: {val_accuracy:.4f}, '
          f'Validation F1 Score: {f1_val:.4f}, '
          f'Learning Rate: {current_lr:.6f}')
    
    train_losses.append(avg_train_loss)
    train_accuracies.append(train_accuracy)
    train_f1_scores.append(f1_train)
    val_losses.append(avg_val_loss)
    val_accuracies.append(val_accuracy)
    val_f1_scores.append(f1_val)
    
  # Early stopping check
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        current_patience = 0
    else:
        current_patience += 1

    if current_patience >= patience:
        print(f'Early stopping after {epoch + 1} epochs without improvement.')
        break

In [ ]:
import matplotlib.pyplot as plt

# Plotting
epochs_range = range(1, num_epochs + 1)

plt.figure(figsize=(15, 5))

# Plot Training and Validation Loss
plt.subplot(1, 3, 1)
plt.plot(epochs_range, train_losses, label='Train Loss')
plt.plot(epochs_range, val_losses, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()

# Plot Training and Validation Accuracy
plt.subplot(1, 3, 2)
plt.plot(epochs_range, train_accuracies, label='Train Accuracy')
plt.plot(epochs_range, val_accuracies, label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()

# Plot Training and Validation F1 Score
plt.subplot(1, 3, 3)
plt.plot(epochs_range, train_f1_scores, label='Train F1 Score')
plt.plot(epochs_range, val_f1_scores, label='Validation F1 Score')
plt.xlabel('Epochs')
plt.ylabel('F1 Score')
plt.title('Training and Validation F1 Score')
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
test_features = pd.read_csv('/kaggle/input/compet/test_features.csv', header=None)
test_features = scaler.transform(test_features)
test_features = pd.DataFrame(test_features, columns=train_features.columns)

In [ ]:
class TestDataset(Dataset):
    def __init__(self, features, transform):
        self.features = features
        self.transform = transform
        self.height = 224
        self.width = 224
        self.channels = 3
        
    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        img_row = self.features.iloc[idx]
        image_array = np.array(img_row).reshape((32, 32))
        image = Image.fromarray(image_array.astype(np.uint8))
        img = image.convert('RGB')
        img_np = np.array(img)
        resized_img = cv2.resize(img_np, (self.width, self.height), interpolation=cv2.INTER_CUBIC)

        # Convert NumPy array back to PIL image
        resized_img_pil = Image.fromarray(resized_img)
        
        tensor = self.transform(resized_img_pil)
        return tensor

In [ ]:
test_dataset = TestDataset(features=test_features, transform=transform)

In [ ]:
batch_size_test = 1
test_loader = DataLoader(test_dataset, batch_size=batch_size_test, shuffle=False)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move the model to the appropriate device
model.to(device)

# Iterate through the test_loader and move input data to the device
with torch.no_grad():
    test_predictions_list = []
    for inputs in test_loader:
        inputs = inputs.to(device)  # Move input data to the same device as the model
        outputs = model(inputs)
        
        # Convert logits to predicted class indices
        _, predicted_test = torch.max(outputs, 1)

        # Move predicted_test to CPU before converting to NumPy
        predicted_test_cpu = predicted_test.cpu().numpy()

        # Append predictions to the list
        test_predictions_list.append(predicted_test_cpu)

# Convert the list of predictions to a numpy array
argmax_predictions = np.concatenate(test_predictions_list)


In [ ]:
submission = pd.DataFrame(data={
    'ID': range(0, len(argmax_predictions)),
    'Prediction': argmax_predictions
})

In [ ]:
submission.to_csv('onestdanslasauce.csv', index=False, header=True)